In [61]:
import pandas as pd
%run C:\Users\sgasmi\Notebooks\Column_autosize.ipynb

In [62]:
ref = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Structure_UO.xlsx") # UO
eff = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx") # Effectif
ref_to_add = ref.copy()

In [63]:
## Modification du fichier effectif
eff = eff[["Matricule","Nom", "Prénom","BU","UO", "Nom de l'ent. juridique", "Matricule Manager Direct","Nom Manager Direct", "Prénom Manager Direct", "Matricule responsable UO", "Nom responsable UO", "Prénom responsabe UO", "Matricule RRH Direct", "Nom RRH Direct", "Prénom RRH Direct", "Matricule RRH d'UO", "Prénom RRH d'UO", "Nom RRH d'UO"]]

In [64]:
ref["Level fille"] = ref["Code type d'unité de tr."].apply(lambda x: x[1:])
ref["Level mère"] = ref.iloc[:, 8]
ref["Level mère"] = ref["Level mère"].fillna("00")
ref["Level mère"] = ref["Level mère"].apply(lambda x: x[1:])
ref["Level mère"] = ref["Level mère"].astype("int64")
ref["Level fille"] = ref["Level fille"].astype("int64")

def check_level(level_fille, level_mere):
    if level_fille>0:
        if (level_fille-1) != level_mere:
            return 1
        else:
            return 0
    else:
        return 0

ref["Anomalie niveau"] = ref.apply(lambda x: check_level(x["Level fille"],x["Level mère"]), axis=1)
ref.loc[ref["Matricule RH"].isna()==True, "RH Manquant"] = 1
ref.loc[ref["Matricule Manager"].isna()==True, "Manager Manquant"] = 1
anomalie_niveau = ref.loc[(ref["Anomalie niveau"]==1) & (ref["Nom de l'unité de travail"] != "Racine")]
anomalie_RH = ref.loc[ref["RH Manquant"]==1]
anomalie_Manager = ref.loc[ref["Manager Manquant"]==1]

In [65]:
if len(anomalie_niveau)>0 or len(anomalie_RH)>0 or len(anomalie_Manager)>0 :
    anomalie_niveau.to_excel(r'C:\Users\sgasmi\Desktop\Export Python\Anomalie UO.xlsx', index=False)
    with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Anomalie UO.xlsx') as writer:
        if len(anomalie_niveau) != 0:
            anomalie_niveau.to_excel(writer, sheet_name='Anomalies_niveau', index=False)
        anomalie_RH.to_excel(writer, sheet_name='Anomalies_RH', index=False)
        anomalie_Manager.to_excel(writer, sheet_name='Anomalie_Manager', index=False)
assert len(anomalie_niveau)==0 , print("Incohérence dans le référentiel")

In [66]:
ref = ref.loc[ref["Code type d'unité de tr."]!="00"] # Suppression du niveau 0
ref.reset_index(drop=True, inplace=True)

In [67]:
col = ["1","2","3","4","5","6","7","8"]
result = pd.DataFrame(columns=col)

In [68]:
print(ref.loc[ref["Code de l'unité de travail"]=="ROOT"])

Empty DataFrame
Columns: [Code de l'unité de travail, Nom de l'unité de travail, Code type d'unité de tr., Objectifs, Date de début, Date de fin, Description, Code unité de tr. mère, Code type d'unité de tr..1, Date de début rattachement, Date de fin rattachement, Code externe UT, Code de l'unité de travail.1, Matricule Manager, Nom Manager, Date de début Manager, Prénom Manager, Matricule RH, Nom RH, Date de début RH, Prénom RH, Level fille, Level mère, Anomalie niveau, RH Manquant, Manager Manquant]
Index: []

[0 rows x 26 columns]


In [69]:
for i, val in enumerate(ref.iterrows()):
    col_cible = ref.at[i, "Code type d'unité de tr."][1]
    # Alimentation de la première colonne
    result.at[i, col_cible] = ref.at[i,"Nom de l'unité de travail"]
    #Alimentation des autres colonnes
    cmere = ""

    while int(col_cible)>1: 
        if cmere== "":
            cmere=ref.at[i,"Code unité de tr. mère"]
        try:
            nuo = ref.loc[ref["Code de l'unité de travail"]==cmere,"Nom de l'unité de travail"].iloc[0]
        except:
            print(ref.at[i,"Nom de l'unité de travail"])
        col_cible= str(int(col_cible)-1)
        result.at[i, col_cible] = nuo
        cmere = ref.loc[ref["Code de l'unité de travail"]==cmere,"Code unité de tr. mère"].iloc[0]
        
    #result.at[i, col_cible-1] = ref.loc[ref["Nom de l'unité de travail"]==ref.at[i,"Code unité de tr. mère"]]
result = result.merge(ref[["Code de l'unité de travail","Nom de l'unité de travail","Code type d'unité de tr.", "Description"]], left_index=True, right_index=True)

In [70]:
# Réorganisation des colonnes
col = result.columns
l1 = list(col[-3:])
l2 = list(list(col[:-3]))
col = l1 + l2

result = result[col]
result.columns=["Nom de l'unité de travail","Code type d'unité de tr.", "Description",
       "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8", "Code de l'unité de travail", ]

result = result[["Code de l'unité de travail","Nom de l'unité de travail", "Code type d'unité de tr.", "Description",
       "L1", "L2", "L3", "L4", "L5", "L6", "L7", "L8"]]

In [71]:
# Ajout de la notion BU
result.loc[result["L2"].str.startswith("OL3")==True, "BU"] = "OL3"
result.loc[(result["L2"].isin(["HR", "QP", "DGP", "ISDT", "3SEP", "RD", "OPX", "OPR"])) |(result["L2"].isna()==True), "BU"] = "FRA-CORP"
result["BU"] = result.apply(lambda x: x["L2"] if str(x["BU"])=="nan" else x["BU"], axis=1)
#result["BU"] = result.apply(lambda x: x["L2"] if x["BU"]=="" else "COCO", axis=1)

### Ajout de l'UO mère

In [72]:
mere = ref[["Code de l'unité de travail", "Code unité de tr. mère"]]
result = result.merge(mere, on="Code de l'unité de travail", how="left")
mere = ref[["Code de l'unité de travail", "Nom de l'unité de travail"]]
result = result.merge(mere, left_on="Code unité de tr. mère", right_on="Code de l'unité de travail", how="left")
result = result.drop(columns=["Code unité de tr. mère", "Code de l'unité de travail_y"], axis=1)
result = result.rename(columns={"Nom de l'unité de travail_y": "UO mère", "Code de l'unité de travail_x":"Code de l'unité de travail", "Nom de l'unité de travail_x": "Nom de l'unité de travail"})
result.loc[result["UO mère"].isna()==True, "UO mère"] = "ROOT"
result = result.sort_values("Code type d'unité de tr.")

In [73]:
result.shape

(1235, 14)

In [74]:
result = result.rename(columns={"Code de l'unité de travail":"Code de l'unité organisationnelle", "Nom de l'unité de travail" : "Nom de l'unité organisationnelle", "Code type d'unité de tr." : "Niveau"})

In [75]:
result =result.merge(ref_to_add[["Code de l'unité de travail", "Matricule Manager", "Nom Manager", "Prénom Manager", "Matricule RH", "Nom RH", "Prénom RH"]], how="left", left_on=["Code de l'unité organisationnelle"], right_on="Code de l'unité de travail")
result = result.rename(columns={"Matricule RH": "Matricule RRH", "Nom RH": "Nom RRH", "Prénom RH": "Prénom RRH"})

In [76]:
eff = eff.sort_values(["BU"])

In [77]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Referentiel UO.xlsx') as writer:
    result.to_excel(writer, sheet_name='Référentiel UO', index=False)
    eff.to_excel(writer, sheet_name='Affectation des salariés', index=False)
    for BU in list(result["BU"].unique()):
        result.loc[result["BU"]==BU].to_excel(writer, sheet_name=BU, index=False)

In [78]:
list_df = []
sheet_names =[]
list_df.append(result)
list_df.append(eff)
sheet_names.append("Référentiel UO")
sheet_names.append('Affectation des salariés')

for BU in list(result["BU"].unique()):
    list_df.append(result.loc[result["BU"]==BU])
    sheet_names.append(str(BU))

In [79]:
resize_column(list_df, sheet_names, r'C:\Users\sgasmi\Desktop\Export Python\Referentiel UO.xlsx' )

Référentiel UO
Affectation des salariés
FRA-CORP
IB
DPIT
FL
OMCA
PCM
DGD
SPIC
SCM
DTI
OL3
